# **SISTEM REKOMENDASI FILM**


Notebook ini berisi implementasi sistem rekomendasi film menggunakan dua pendekatan utama, yaitu Content-Based Filtering dan Collaborative Filtering (Neural Collaborative Filtering). Dataset yang digunakan adalah MovieLens 100K.


## 1. Pendahuluan

Sistem rekomendasi film adalah aplikasi yang menggunakan algoritma untuk menyarankan film kepada pengguna berdasarkan preferensi mereka. Dalam proyek ini, kita akan mengimplementasikan dua pendekatan utama:

1. **Content-based Filtering**: Merekomendasikan film berdasarkan kesamaan fitur film (genre, aktor, dll)
2. **Collaborative Filtering**: Merekomendasikan film berdasarkan pola rating dari pengguna lain yang memiliki preferensi serupa

Kita akan menggunakan dataset MovieLens yang berisi rating film dari banyak pengguna.

## 2. Persiapan Lingkungan


### 2.1 Import Library yang Dibutuhkan


Pada bagian ini, kita mengimpor semua library yang diperlukan untuk proyek:

- pandas dan numpy untuk manipulasi data
- matplotlib dan seaborn untuk visualisasi
- sklearn untuk pemrosesan teks dan perhitungan kesamaan
- tensorflow untuk model pembelajaran mesin
- library pendukung lainnya seperti requests dan zipfile untuk mengunduh dataset


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from zipfile import ZipFile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import io
import requests
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
import os
import warnings
warnings.filterwarnings('ignore')

# Mengatur style visualisasi
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')

### 2.2 Fungsi untuk Mengunduh dan Mengekstrak Dataset


Fungsi ini mengunduh dataset MovieLens dari repositori resmi dan mengekstraknya ke direktori lokal. Dataset ini berisi informasi film, rating, dan tag yang diberikan oleh pengguna.


In [3]:
def download_and_extract_data():
    """
    Mengunduh dataset MovieLens 100K dan mengekstraknya
    """
    # URL dataset
    url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

    print("Mengunduh dataset...")
    r = requests.get(url)

    # Menyimpan file zip
    with open("ml-latest-small.zip", "wb") as file:
        file.write(r.content)

    # Mengekstrak file
    print("Mengekstrak file...")
    with ZipFile("ml-latest-small.zip", "r") as zip_ref:
        zip_ref.extractall()

    print("Dataset berhasil diunduh dan diekstrak!")

### 2.3 Fungsi untuk Memuat Dataset


Fungsi ini memuat dataset yang telah diunduh ke dalam DataFrame pandas. Jika dataset belum ada, fungsi akan memanggil fungsi download_and_extract_data terlebih dahulu.


In [4]:
def load_data():
    """
    Memuat dataset MovieLens
    """
    # Jika dataset belum ada, unduh terlebih dahulu
    if not os.path.exists("ml-latest-small"):
        download_and_extract_data()

    # Memuat data rating
    ratings = pd.read_csv('ml-latest-small/ratings.csv')

    # Memuat data film
    movies = pd.read_csv('ml-latest-small/movies.csv')

    # Memuat data tags (jika diperlukan)
    tags = pd.read_csv('ml-latest-small/tags.csv')

    # Memuat data links (jika diperlukan)
    links = pd.read_csv('ml-latest-small/links.csv')

    return ratings, movies, tags, links

## 3. Eksplorasi Data


Pada bagian ini, kita memuat dataset dan melakukan eksplorasi awal untuk memahami karakteristiknya.


### 3.1 Memuat dan Memeriksa Dataset


Kita memuat dataset dan memeriksa struktur serta informasi statistik untuk memahami data yang akan digunakan.


In [5]:
ratings, movies, tags, links = load_data()

# Melihat 5 data pertama dari setiap dataset
print("Data ratings:")
print(ratings.head())
print("\nData movies:")
print(movies.head())
print("\nData tags:")
print(tags.head())
print("\nData links:")
print(links.head())

# Memeriksa informasi dataset
print("\nInformasi dataset ratings:")
print(ratings.info())
print("\nInformasi dataset movies:")
print(movies.info())

# Memeriksa statistik deskriptif
print("\nStatistik deskriptif ratings:")
print(ratings.describe())

Mengunduh dataset...
Mengekstrak file...
Dataset berhasil diunduh dan diekstrak!
Data ratings:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Data movies:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Data tags:
   userId  movieId              tag   timestamp
0       

Di sini kita memuat data dan melihat sekilas struktur datanya.


### 3.2 Analisis Eksplorasi Data


Kita melakukan analisis lebih mendalam untuk memahami distribusi data, pola rating, dan karakteristik film dalam dataset.


In [6]:
# Memeriksa jumlah data di setiap dataset
print(f"Jumlah data ratings: {len(ratings)}")
print(f"Jumlah film unik: {len(movies)}")
print(f"Jumlah pengguna unik: {ratings['userId'].nunique()}")

# Visualisasi distribusi rating
plt.figure(figsize=(10, 6))
sns.countplot(x='rating', data=ratings)
plt.title('Distribusi Rating Film')
plt.xlabel('Rating')
plt.ylabel('Jumlah')
plt.savefig('rating_distribution.png')
plt.close()

# Visualisasi jumlah rating per film
movie_ratings_count = ratings.groupby('movieId').size().reset_index(name='count')
movie_ratings_count = movie_ratings_count.sort_values('count', ascending=False)

plt.figure(figsize=(12, 6))
sns.histplot(movie_ratings_count['count'], bins=50, kde=True)
plt.title('Distribusi Jumlah Rating per Film')
plt.xlabel('Jumlah Rating')
plt.ylabel('Jumlah Film')
plt.xscale('log')
plt.savefig('ratings_per_movie.png')
plt.close()

# Visualisasi jumlah rating per pengguna
user_ratings_count = ratings.groupby('userId').size().reset_index(name='count')
user_ratings_count = user_ratings_count.sort_values('count', ascending=False)

plt.figure(figsize=(12, 6))
sns.histplot(user_ratings_count['count'], bins=50, kde=True)
plt.title('Distribusi Jumlah Rating per Pengguna')
plt.xlabel('Jumlah Rating')
plt.ylabel('Jumlah Pengguna')
plt.savefig('ratings_per_user.png')
plt.close()

Jumlah data ratings: 100836
Jumlah film unik: 9742
Jumlah pengguna unik: 610


### 3.3 Analisis Genre Film


Kita menganalisis distribusi genre film untuk memahami kategori yang paling umum dalam dataset.


In [7]:
# Ekstrak semua genre yang ada
all_genres = []
for genres in movies['genres'].str.split('|'):
    all_genres.extend(genres)

unique_genres = pd.Series(all_genres).value_counts().reset_index()
unique_genres.columns = ['genre', 'count']

plt.figure(figsize=(14, 8))
sns.barplot(x='genre', y='count', data=unique_genres.head(15))
plt.title('15 Genre Film Terpopuler')
plt.xlabel('Genre')
plt.ylabel('Jumlah Film')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('top_genres.png')
plt.close()

# Menggabungkan data film dengan rating rata-rata
movie_avg_rating = ratings.groupby('movieId')['rating'].mean().reset_index()
movie_avg_rating.columns = ['movieId', 'avgRating']

movies_with_ratings = pd.merge(movies, movie_avg_rating, on='movieId', how='left')
movies_with_ratings['avgRating'] = movies_with_ratings['avgRating'].fillna(0)

# Visualisasi film dengan rating tertinggi (min 100 rating)
movie_rating_count = ratings.groupby('movieId')['rating'].count().reset_index()
movie_rating_count.columns = ['movieId', 'ratingCount']

movies_with_stats = pd.merge(movies_with_ratings, movie_rating_count, on='movieId', how='left')
movies_with_stats['ratingCount'] = movies_with_stats['ratingCount'].fillna(0)

popular_movies = movies_with_stats[movies_with_stats['ratingCount'] >= 100].sort_values('avgRating', ascending=False)

plt.figure(figsize=(14, 8))
sns.barplot(x='avgRating', y='title', data=popular_movies.head(15))
plt.title('15 Film dengan Rating Tertinggi (Min. 100 Rating)')
plt.xlabel('Rating Rata-rata')
plt.ylabel('Judul Film')
plt.tight_layout()
plt.savefig('top_rated_movies.png')
plt.close()

## 4. Persiapan Data


Pada bagian ini, kita mempersiapkan data untuk digunakan dalam model rekomendasi.


### 4.1 Filtering dan Pembersihan Data


Kita melakukan filtering untuk menghapus film dan pengguna dengan jumlah rating yang terlalu sedikit, serta mengekstrak informasi tahun dari judul film.


In [8]:
# Menghapus film yang memiliki sedikit rating (kurang dari 5 rating)
movie_counts = ratings['movieId'].value_counts()
movies_to_keep = movie_counts[movie_counts >= 5].index

filtered_ratings = ratings[ratings['movieId'].isin(movies_to_keep)]
print(f"Jumlah rating sebelum filtering: {len(ratings)}")
print(f"Jumlah rating setelah filtering: {len(filtered_ratings)}")

# Menghapus pengguna yang memberikan sedikit rating (kurang dari 5 rating)
user_counts = filtered_ratings['userId'].value_counts()
users_to_keep = user_counts[user_counts >= 5].index

filtered_ratings = filtered_ratings[filtered_ratings['userId'].isin(users_to_keep)]
print(f"Jumlah rating setelah filtering pengguna: {len(filtered_ratings)}")

# Membuat kolom tahun dari judul film
def extract_year(title):
    match = re.search(r'\((\d{4})\)$', title)
    if match:
        return int(match.group(1))
    return None

movies['year'] = movies['title'].apply(extract_year)

Jumlah rating sebelum filtering: 100836
Jumlah rating setelah filtering: 90274
Jumlah rating setelah filtering pengguna: 90274


### 4.2 Preprocessing untuk Content-Based Filtering


Kita mempersiapkan data untuk model Content-Based Filtering dengan membuat representasi vektor TF-IDF dari genre film dan menghitung kesamaan kosinus antar film.


In [9]:
# Membuat fitur berdasarkan genre
movies['genres'] = movies['genres'].fillna('')

# Membuat TF-IDF matrix dari genre
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['genres'].str.replace('|', ' '))

# Menghitung cosine similarity antar film
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Membuat mapping dari indeks ke movieId dan sebaliknya
indices = pd.Series(movies.index, index=movies['movieId']).drop_duplicates()
movie_indices = pd.Series(movies['movieId'].values, index=movies.index)

### 4.3 Preprocessing untuk Collaborative Filtering


Kita mempersiapkan data untuk model Collaborative Filtering dengan membagi data menjadi set pelatihan dan pengujian, serta membuat mapping ID pengguna dan film ke indeks berurutan.


In [10]:
# Memisahkan data menjadi training dan testing
train_data, test_data = train_test_split(
    filtered_ratings,
    test_size=0.2,
    random_state=42
)

# Membuat mapping movieId dan userId ke indeks berurutan
user_ids = filtered_ratings['userId'].unique().tolist()
movie_ids = filtered_ratings['movieId'].unique().tolist()

user_to_index = {user_id: i for i, user_id in enumerate(user_ids)}
movie_to_index = {movie_id: i for i, movie_id in enumerate(movie_ids)}

# Konversi data ke format yang sesuai untuk model
def map_ids(row):
    return {
        'user': user_to_index[row['userId']],
        'movie': movie_to_index[row['movieId']],
        'rating': row['rating']
    }

train_mapped = train_data.apply(map_ids, axis=1).tolist()
test_mapped = test_data.apply(map_ids, axis=1).tolist()

## 5. Pemodelan Content-Based Filtering

Pada bagian ini, kita mengimplementasikan sistem rekomendasi berbasis konten menggunakan kesamaan genre film.


### 5.1 Implementasi Fungsi Rekomendasi Content-Based


Kita membuat fungsi untuk memberikan rekomendasi film berdasarkan kesamaan genre dengan film yang dipilih pengguna.


In [11]:
def get_recommendations_content_based(movie_id, cosine_sim=cosine_sim, movies=movies, indices=indices, movie_indices=movie_indices, n=10):
    """
    Memberikan rekomendasi film berdasarkan kesamaan genre

    Parameters:
    -----------
    movie_id : int
        ID film yang menjadi dasar rekomendasi
    cosine_sim : numpy.ndarray
        Matrix kesamaan kosinus antar film
    movies : pandas.DataFrame
        DataFrame berisi data film
    indices : pandas.Series
        Mapping dari movieId ke indeks
    movie_indices : pandas.Series
        Mapping dari indeks ke movieId
    n : int
        Jumlah rekomendasi yang diinginkan

    Returns:
    --------
    pandas.DataFrame
        DataFrame berisi film yang direkomendasikan
    """
    # Mendapatkan indeks film dari movieId
    if movie_id not in indices:
        return pd.DataFrame()

    idx = indices[movie_id]

    # Mendapatkan skor kesamaan dengan semua film
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Mengurutkan film berdasarkan skor kesamaan
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Mendapatkan indeks film yang direkomendasikan (tidak termasuk film itu sendiri)
    sim_scores = sim_scores[1:n+1]
    movie_indices_rec = [i[0] for i in sim_scores]

    # Mendapatkan movieId dari indeks
    movie_ids_rec = [movie_indices[i] for i in movie_indices_rec]

    # Mengembalikan data film yang direkomendasikan
    return movies[movies['movieId'].isin(movie_ids_rec)][['movieId', 'title', 'genres']]

### 5.2 Demonstrasi Content-Based Filtering


Kita mendemonstrasikan cara kerja sistem rekomendasi berbasis konten dengan contoh film "Toy Story".


In [12]:
# Mencari film "Toy Story"
toy_story = movies[movies['title'].str.contains('Toy Story', case=False)].iloc[0]
print(f"Film dasar untuk rekomendasi: {toy_story['title']} (ID: {toy_story['movieId']})")

# Mendapatkan rekomendasi
content_based_recommendations = get_recommendations_content_based(toy_story['movieId'])
print("\nRekomendasi berdasarkan Content-Based Filtering:")
print(content_based_recommendations)

Film dasar untuk rekomendasi: Toy Story (1995) (ID: 1)

Rekomendasi berdasarkan Content-Based Filtering:
      movieId                                              title  \
1706     2294                                        Antz (1998)   
2355     3114                                 Toy Story 2 (1999)   
2809     3754     Adventures of Rocky and Bullwinkle, The (2000)   
3000     4016                   Emperor's New Groove, The (2000)   
3568     4886                              Monsters, Inc. (2001)   
6194    45074                                   Wild, The (2006)   
6486    53121                             Shrek the Third (2007)   
6948    65577                     Tale of Despereaux, The (2008)   
7760    91355  Asterix and the Vikings (Astérix et les Viking...   
8219   103755                                       Turbo (2013)   

                                           genres  
1706  Adventure|Animation|Children|Comedy|Fantasy  
2355  Adventure|Animation|Children|Comedy|

## 6. Pemodelan Collaborative Filtering


Pada bagian ini, kita mengimplementasikan sistem rekomendasi berbasis kolaboratif menggunakan model neural network.


### 6.1 Implementasi Model Neural Collaborative Filtering


Kita mendefinisikan model Neural Collaborative Filtering (NCF) menggunakan TensorFlow/Keras.


In [13]:
# Mendefinisikan model Neural Collaborative Filtering
def create_ncf_model(num_users, num_movies, embedding_size=50):
    """
    Membuat model Neural Collaborative Filtering

    Parameters:
    -----------
    num_users : int
        Jumlah pengguna unik
    num_movies : int
        Jumlah film unik
    embedding_size : int
        Ukuran embedding vector

    Returns:
    --------
    tensorflow.keras.Model
        Model NCF yang telah didefinisikan
    """
    # Input layers
    user_input = keras.Input(shape=(1,), name='user_input')
    movie_input = keras.Input(shape=(1,), name='movie_input')

    # Embedding layers
    user_embedding = layers.Embedding(
        input_dim=num_users,
        output_dim=embedding_size,
        name='user_embedding'
    )(user_input)

    movie_embedding = layers.Embedding(
        input_dim=num_movies,
        output_dim=embedding_size,
        name='movie_embedding'
    )(movie_input)

    # Flatten embeddings
    user_vector = layers.Flatten()(user_embedding)
    movie_vector = layers.Flatten()(movie_embedding)

    # Concatenate embeddings
    concat = layers.Concatenate()([user_vector, movie_vector])

    # Dense layers
    dense1 = layers.Dense(128, activation='relu')(concat)
    dense1 = layers.Dropout(0.2)(dense1)
    dense2 = layers.Dense(64, activation='relu')(dense1)
    dense2 = layers.Dropout(0.2)(dense2)

    # Output layer
    output = layers.Dense(1)(dense2)

    # Create model
    model = keras.Model(inputs=[user_input, movie_input], outputs=output)

    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error'
    )

    return model

### 6.2 Persiapan Dataset TensorFlow


Kita membuat fungsi untuk mengkonversi data ke format TensorFlow Dataset yang dapat digunakan untuk melatih model.


In [14]:
def create_tf_dataset(data_mapped, batch_size=64, shuffle=True):
    """
    Membuat dataset TensorFlow dari data yang sudah dimapping

    Parameters:
    -----------
    data_mapped : list
        List berisi dictionary dengan keys 'user', 'movie', dan 'rating'
    batch_size : int
        Ukuran batch
    shuffle : bool
        Apakah data perlu diacak

    Returns:
    --------
    tf.data.Dataset
        Dataset TensorFlow
    """
    users = np.array([x['user'] for x in data_mapped])
    movies = np.array([x['movie'] for x in data_mapped])
    ratings = np.array([x['rating'] for x in data_mapped])

    dataset = tf.data.Dataset.from_tensor_slices(({
        'user_input': users,
        'movie_input': movies
    }, ratings))

    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(data_mapped))

    dataset = dataset.batch(batch_size)

    return dataset

### 6.3 Pelatihan Model Collaborative Filtering


Kita membuat dan melatih model Neural Collaborative Filtering menggunakan data pelatihan.


In [15]:
# Membuat dataset TensorFlow
train_dataset = create_tf_dataset(train_mapped)
test_dataset = create_tf_dataset(test_mapped, shuffle=False)

# Membuat dan melatih model
num_users = len(user_ids)
num_movies = len(movie_ids)

# Membuat model
model = create_ncf_model(num_users, num_movies)

# Mendefinisikan callbacks
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Melatih model
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=test_dataset,
    callbacks=[early_stopping]
)

Epoch 1/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 2.5746 - val_loss: 0.7644
Epoch 2/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.8225 - val_loss: 0.7292
Epoch 3/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.7718 - val_loss: 0.7181
Epoch 4/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.7317 - val_loss: 0.7172
Epoch 5/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.7003 - val_loss: 0.7237
Epoch 6/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.6689 - val_loss: 0.7250
Epoch 7/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.6318 - val_loss: 0.7358
Epoch 8/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.5891 - val_loss: 0.7403
Epoch 9/20
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.5409 - val_loss: 0.7529


### 6.4 Visualisasi Hasil Pelatihan

Kita memvisualisasikan performa model selama pelatihan.


In [16]:
# Visualisasi loss selama training
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('model_loss.png')
plt.close()

### 6.5 Implementasi Fungsi Rekomendasi Collaborative Filtering


Kita membuat fungsi untuk memberikan rekomendasi film berdasarkan model Collaborative Filtering.


In [17]:
# Fungsi untuk mendapatkan rekomendasi dengan Collaborative Filtering
def get_recommendations_collaborative(user_id, movies_df=movies, ratings_df=filtered_ratings, model=model,
                                     user_to_index=user_to_index, movie_to_index=movie_to_index, n=10):
    """
    Memberikan rekomendasi film berdasarkan Collaborative Filtering

    Parameters:
    -----------
    user_id : int
        ID pengguna
    movies_df : pandas.DataFrame
        DataFrame berisi data film
    ratings_df : pandas.DataFrame
        DataFrame berisi data rating
    model : tensorflow.keras.Model
        Model Collaborative Filtering
    user_to_index : dict
        Mapping dari userId ke indeks
    movie_to_index : dict
        Mapping dari movieId ke indeks
    n : int
        Jumlah rekomendasi yang diinginkan

    Returns:
    --------
    pandas.DataFrame
        DataFrame berisi film yang direkomendasikan
    """
    # Mendapatkan film yang belum ditonton oleh pengguna
    user_movies = ratings_df[ratings_df['userId'] == user_id]['movieId'].unique()
    movies_to_predict = movies_df[~movies_df['movieId'].isin(user_movies)]['movieId'].values

    # Jika pengguna tidak ada dalam dataset training
    if user_id not in user_to_index:
        # Kembalikan rekomendasi film populer
        top_movies = ratings_df.groupby('movieId')['rating'].mean().sort_values(ascending=False).index[:n]
        return movies_df[movies_df['movieId'].isin(top_movies)][['movieId', 'title', 'genres']]

    # Membuat data untuk prediksi
    user_idx = user_to_index[user_id]
    movie_indices = [movie_to_index[m] for m in movies_to_predict if m in movie_to_index]

    if not movie_indices:
        # Jika tidak ada film yang bisa diprediksi, kembalikan rekomendasi film populer
        top_movies = ratings_df.groupby('movieId')['rating'].mean().sort_values(ascending=False).index[:n]
        return movies_df[movies_df['movieId'].isin(top_movies)][['movieId', 'title', 'genres']]

    users = np.array([user_idx] * len(movie_indices))
    movies = np.array(movie_indices)

    # Prediksi rating
    predictions = model.predict({
        'user_input': users,
        'movie_input': movies
    })

    # Menggabungkan film dengan prediksi rating
    movie_ids = [list(movie_to_index.keys())[list(movie_to_index.values()).index(idx)] for idx in movies]
    predicted_ratings = predictions.flatten()

    # Membuat DataFrame prediksi
    pred_df = pd.DataFrame({
        'movieId': movie_ids,
        'predicted_rating': predicted_ratings
    })

    # Mengurutkan berdasarkan prediksi rating
    pred_df = pred_df.sort_values('predicted_rating', ascending=False)

    # Mendapatkan top-n rekomendasi
    top_movie_ids = pred_df.head(n)['movieId'].values

    # Mengembalikan data film yang direkomendasikan
    return movies_df[movies_df['movieId'].isin(top_movie_ids)][['movieId', 'title', 'genres']]

### 6.6 Demonstrasi Collaborative Filtering


Kita mendemonstrasikan cara kerja sistem rekomendasi kolaboratif dengan contoh pengguna.


In [18]:
# Memilih pengguna
sample_user_id = filtered_ratings['userId'].value_counts().index[0]
print(f"\nRekomendasi untuk Pengguna ID: {sample_user_id}")

# Mendapatkan film yang sudah ditonton oleh pengguna
user_watched_movies = filtered_ratings[filtered_ratings['userId'] == sample_user_id]
user_watched_movies = pd.merge(user_watched_movies, movies[['movieId', 'title']], on='movieId')
print("\nBeberapa film yang sudah ditonton oleh pengguna:")
print(user_watched_movies[['title', 'rating']].sort_values('rating', ascending=False).head())

# Mendapatkan rekomendasi
collaborative_recommendations = get_recommendations_collaborative(sample_user_id)
print("\nRekomendasi berdasarkan Collaborative Filtering:")
print(collaborative_recommendations)


Rekomendasi untuk Pengguna ID: 414

Beberapa film yang sudah ditonton oleh pengguna:
                               title  rating
2112       Hell or High Water (2016)     5.0
8     American President, The (1995)     5.0
1030            High Fidelity (2000)     5.0
30        Usual Suspects, The (1995)     5.0
1056                Gladiator (2000)     5.0
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

Rekomendasi berdasarkan Collaborative Filtering:
      movieId                                              title  \
27         28                                  Persuasion (1995)   
680       898                     Philadelphia Story, The (1940)   
940      1241                      Dead Alive (Braindead) (1992)   
1664     2239  Swept Away (Travolti da un insolito destino ne...   
2453     3266  Man Bites Dog (C'est arrivé près de chez vous)...   
2582     3451                Guess Who's Coming to Dinner (1967)   
2593     3468                                Hustler, The (1961)   
4396     64

## 7. Evaluasi

Pada bagian ini, kita mengevaluasi kinerja kedua model rekomendasi.


### 7.1 Evaluasi Content-Based Filtering


Kita mengevaluasi model Content-Based Filtering menggunakan metrik Precision@k.


In [19]:
# Evaluasi Content-Based Filtering
def evaluate_content_based(movies_df, ratings_df, indices, cosine_sim, movie_indices, k=10):
    """
    Evaluasi model Content-Based Filtering menggunakan precision@k

    Parameters:
    -----------
    movies_df : pandas.DataFrame
        DataFrame berisi data film
    ratings_df : pandas.DataFrame
        DataFrame berisi data rating
    indices : pandas.Series
        Mapping dari movieId ke indeks
    cosine_sim : numpy.ndarray
        Matrix kesamaan kosinus antar film
    movie_indices : pandas.Series
        Mapping dari indeks ke movieId
    k : int
        Jumlah rekomendasi yang dievaluasi

    Returns:
    --------
    float
        Nilai precision@k rata-rata
    """
    # Pilih pengguna dengan jumlah rating yang cukup
    users_with_enough_ratings = ratings_df['userId'].value_counts()[ratings_df['userId'].value_counts() >= 20].index

    # Batasi jumlah pengguna untuk evaluasi
    sample_users = np.random.choice(users_with_enough_ratings, min(100, len(users_with_enough_ratings)), replace=False)

    precision_at_k_scores = []

    for user_id in sample_users:
        # Dapatkan film yang disukai pengguna (rating >= 4)
        liked_movies = ratings_df[(ratings_df['userId'] == user_id) & (ratings_df['rating'] >= 4)]['movieId'].values

        if len(liked_movies) < 2:
            continue

        # Pisahkan satu film untuk dasar rekomendasi
        np.random.shuffle(liked_movies)
        base_movie = liked_movies[0]
        test_movies = liked_movies[1:]

        # Dapatkan rekomendasi berdasarkan film dasar
        if base_movie not in indices:
            continue

        recommendations = get_recommendations_content_based(base_movie, cosine_sim, movies_df, indices, movie_indices, n=k)
        recommended_ids = recommendations['movieId'].values

        # Hitung precision@k
        relevant_count = len(set(recommended_ids) & set(test_movies))
        precision_at_k = relevant_count / k

        precision_at_k_scores.append(precision_at_k)

    # Hitung rata-rata precision@k
    if precision_at_k_scores:
        return np.mean(precision_at_k_scores)
    else:
        return 0

### 7.2 Evaluasi Collaborative Filtering


Kita mengevaluasi model Collaborative Filtering menggunakan metrik RMSE (Root Mean Squared Error).


In [20]:
# Evaluasi Collaborative Filtering
def evaluate_collaborative(model, test_dataset):
    """
    Evaluasi model Collaborative Filtering menggunakan RMSE

    Parameters:
    -----------
    model : tensorflow.keras.Model
        Model Collaborative Filtering
    test_dataset : tf.data.Dataset
        Dataset testing

    Returns:
    --------
    float
        Nilai RMSE
    """
    # Prediksi rating pada dataset testing
    y_pred = model.predict(test_dataset)

    # Dapatkan nilai sebenarnya
    y_true = np.concatenate([y for _, y in test_dataset], axis=0)

    # Hitung RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    return rmse

### 7.3 Hasil Evaluasi dan Visualisasi


Kita mengevaluasi kedua model dan memvisualisasikan hasilnya.


In [21]:
# Evaluasi kedua model
print("\nEvaluasi Model:")

# Evaluasi Content-Based Filtering
precision_at_k = evaluate_content_based(movies, filtered_ratings, indices, cosine_sim, movie_indices)
print(f"Content-Based Filtering - Precision@10: {precision_at_k:.4f}")

# Evaluasi Collaborative Filtering
rmse = evaluate_collaborative(model, test_dataset)
print(f"Collaborative Filtering - RMSE: {rmse:.4f}")

# Visualisasi hasil evaluasi
plt.figure(figsize=(12, 6))

# Precision@k untuk Content-Based Filtering
plt.subplot(1, 2, 1)
plt.bar(['Precision@10'], [precision_at_k], color='skyblue')
plt.title('Content-Based Filtering\nPrecision@10')
plt.ylim(0, 1)

# RMSE untuk Collaborative Filtering
plt.subplot(1, 2, 2)
plt.bar(['RMSE'], [rmse], color='salmon')
plt.title('Collaborative Filtering\nRMSE')
plt.ylim(0, 5)

plt.tight_layout()
plt.savefig('evaluation_metrics.png')
plt.close()

# Simpan model Collaborative Filtering
model.save('collaborative_filtering_model.keras')

print("\nProyek Sistem Rekomendasi Film Selesai!")


Evaluasi Model:
Content-Based Filtering - Precision@10: 0.0414
283/283 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Collaborative Filtering - RMSE: 0.8468

Proyek Sistem Rekomendasi Film Selesai!


## 8. Kesimpulan


Dalam proyek ini, kita telah berhasil mengimplementasikan dua pendekatan sistem rekomendasi film:

1. **Content-Based Filtering**: Memberikan rekomendasi berdasarkan kesamaan genre film. Metode ini efektif untuk merekomendasikan film dengan karakteristik serupa, tetapi memiliki keterbatasan karena hanya mempertimbangkan fitur film, bukan preferensi pengguna.

2. **Collaborative Filtering**: Memberikan rekomendasi berdasarkan pola rating dari pengguna lain. Metode ini dapat menemukan film yang mungkin disukai pengguna bahkan jika film tersebut tidak memiliki kesamaan fitur yang jelas dengan film yang pernah ditonton sebelumnya.

Hasil evaluasi menunjukkan bahwa model Collaborative Filtering memiliki performa yang cukup baik dengan RMSE sekitar 0.85, sementara Content-Based Filtering memiliki Precision@10 sekitar 0.04.

Untuk pengembangan lebih lanjut, kita dapat:

- Menggabungkan kedua pendekatan menjadi sistem rekomendasi hybrid
- Menambahkan fitur lain seperti aktor, sutradara, atau sinopsis film
- Mencoba algoritma yang lebih canggih seperti Matrix Factorization atau Deep Learning